# This notebook plots a map image of WRF-Hydro NetCDF grids

In [131]:
# turn on interactive plotting
%matplotlib notebook  

In [132]:
# turn off warnings for deprecated code in matplotlib 
import warnings
warnings.filterwarnings('ignore')

In [133]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import mplleaflet
import pandas as pd

In [134]:
model_in = 'D:/Jupyter_Notebooks/Testing/data/geo_em.d01.boulder_creek_1km.nc'
model_out = 'D:/Jupyter_Notebooks/Testing/data/2014081216_LDASOUT_DOMAIN1'
geo = Dataset(model_in, mode='r')
nc = Dataset(model_out, mode='r')

In [135]:
geo.variables.keys()

odict_keys(['Times', 'XLAT_M', 'XLONG_M', 'XLAT_V', 'XLONG_V', 'XLAT_U', 'XLONG_U', 'CLAT', 'CLONG', 'MAPFAC_M', 'MAPFAC_V', 'MAPFAC_U', 'MAPFAC_MX', 'MAPFAC_VX', 'MAPFAC_UX', 'MAPFAC_MY', 'MAPFAC_VY', 'MAPFAC_UY', 'E', 'F', 'SINALPHA', 'COSALPHA', 'LANDMASK', 'LANDUSEF', 'LU_INDEX', 'HGT_M', 'SLPX', 'SLPY', 'SOILTEMP', 'SOILCTOP', 'SCT_DOM', 'SOILCBOT', 'SCB_DOM', 'ALBEDO12M', 'GREENFRAC', 'SNOALB', 'SLOPECAT', 'CON', 'VAR', 'OA1', 'OA2', 'OA3', 'OA4', 'OL1', 'OL2', 'OL3', 'OL4', 'VAR_SSO'])

In [136]:
nc.variables.keys()
#nc.variables

odict_keys(['Times', 'IVGTYP', 'ISLTYP', 'SKINTEMP', 'CANWAT', 'SOIL_T', 'SOIL_M', 'SOIL_W', 'SOIL_MX', 'SFCRNOFF', 'UGDRNOFF', 'INTRFLOW', 'SFCEVP', 'ETPND', 'ETAKIN', 'CANEVP', 'EDIRX', 'ETTX', 'ALBEDX', 'WEASD', 'ACRAIN', 'ACSNOM', 'ESNOW', 'DRIP', 'DEWFALL', 'SNODEP', 'VEGFRA', 'Z0', 'HFX', 'QFX', 'GRDFLX', 'SW', 'LW', 'FDOWN', 'XLAI', 'SNOTIME', 'EMBRD', 'SNOALB', 'NOAHRES', 'CH'])

In [137]:
lats = geo.variables['XLAT_M'][0,::-1,:] # these are 3D arrays and needed to be reduced to 2d
lons = geo.variables['XLONG_M'][0,::-1,:]
#luse = np.array(nc.variables['LANDUSEF'][0,::-1,:]) # land use is multidimensional (each cat is a dimension: 24)
#df = pd.DataFrame(lats)
#ax = df.plot()

In [ ]:
lon_0 = lons.mean()
lat_0 = lats.mean()
lat_min = lats.min()
lon_min = lons.min()
lat_max = lats.max()
lon_max = lons.max()
print(str(lon_min)+','+str(lat_min)+','+str(lon_max)+','+str(lat_max))

-105.718,39.7833,-105.144,40.096


In [ ]:
m = Basemap(width=5000000,height=3500000,
            resolution='i',epsg=26954,\
            lat_ts=40,lat_0=lat_0,lon_0=lon_0, llcrnrlon=lon_min,llcrnrlat=lat_min,urcrnrlon=lon_max,urcrnrlat=lat_max)

In [ ]:
#lon, lat = np.meshgrid(lons, lats)
xi, yi = m(lons, lats)

In [ ]:
var = 'SFCRNOFF' # Enter the variable id to plot
var_data = np.array(nc.variables[var][0,::-1,:])
var_units = nc.variables[var].units
var_desc = nc.variables[var].description

# mask invalid data
var_plot = np.ma.masked_less(var_data, -9999)

# infor about variable to be plotted
nc.variables[var]

In [ ]:
# Plot desired grid
cs = m.pcolormesh(xi,yi,var_plot)

In [ ]:
nc.close()

In [ ]:
plt.savefig('netcdf_image_out.png',bbox_inches='tight')

In [ ]:
import folium
from folium import plugins
import mplleaflet
print(folium.__version__)

In [ ]:
def inline_map(m):
    from folium import Map
    from IPython.display import HTML, IFrame
    if isinstance(m, Map):
        m._build_map()
        srcdoc = m.HTML.replace('"', '&quot;')
        embed = HTML('<iframe srcdoc="{srcdoc}" '
                     'style="width: 100%; height: 500px; '
                     'border: none"></iframe>'.format(srcdoc=srcdoc))
    elif isinstance(m, str):
        embed = IFrame(m, width=750, height=500)
    return embed

In [ ]:
mf = folium.Map(location=[39.9, -105.38], tiles='Stamen Terrain',
                   zoom_start=10, control_scale=True, prefer_canvas=True)

In [ ]:
outfp = "folium_test_map.html"

In [ ]:
img = plugins.ImageOverlay(name=str(var_desc)+' '+str(var_units),image='netcdf_image_out.png',bounds=[[float(lat_min), float(lon_min)], [float(lat_max), float(lon_max)]],
                          opacity=0.5,interactive=True,pixelated=True,control=True)

In [ ]:
usdm= folium.features.WmsTileLayer("http://ndmc-001.unl.edu:8080/cgi-bin/mapserv.exe?map=/ms4w/apps/usdm/service/usdm_current_wms.map&SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=usdm_current&WIDTH=640&HEIGHT=480&crs=EPSG:3857&styles=default&format=image/png&bbox=-18367715.9809,1689200.13961,-6679169.4476,15538711.0963", 
                    layers="usdm_current",transparent=True, format="image/png",name="USDM",version="1.3",overlay=True, control=True)
vegdri= folium.features.WmsTileLayer("https://vegdri.cr.usgs.gov/wms.php?TRANSPARENT=true&PROJECTION=EPSG%3A900913&UNITS=m&LAYERS=DROUGHT_VDRI_EMODIS_1&MAXEXTENT=NaN%2C%2C%2C&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&STYLES=qdri_vegdri_raster&FORMAT=image%2Fpng&SRS=EPSG%3A900913&BBOX=-15812679.213931,1442518.8685938,-5187320.7860688,7557481.1314062&WIDTH=1086&HEIGHT=625", 
                    layers="DROUGHT_VDRI_EMODIS_1",transparent=True, format="image/png",version="1.1.1",overlay=True, control=True)

In [ ]:
#mf.add_children(usdm)
#mf.add_children(vegdri)
mf.add_children(img)
mf.add_children(folium.map.LayerControl())
mf.save(outfp)

In [ ]:
mf

In [ ]:
inline_map("folium_test_map.html")